Install MonsterAPI pypi client

In [ ]:
# Please install specific beta version of client for quick serve access.
!python3 -m pip install monsterapi==1.0.2b3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.7 MB/s eta 0:00:00


Sign up on [MonsterAPI](https://monsterapi.ai/signup?utm_source=llama-index-colab&utm_medium=referral) and get a free auth key. Paste it below:
Make sure you have signed up  for beta access at [here](https://forms.gle/TTJRapHm59RxjttJA)

In [ ]:
api_key = "<replace with your api key>"

## Initialize client

In [ ]:
from monsterapi import client as mclient
deploy_client = mclient(api_key = api_key)

## Launch a LLM QuickServe deployment
Let quick serve Code LLama with lora adapter from hugging face. Craeate a deployment on a 24GB RTXA5000 GPU.
This RestAPI can serve 380 Tokens/Sec through REST API with both static and streaming response support.

```
    basemodel_path="codellama/CodeLlama-7b-hf"
    loramodel_path="qblocks/codellama7b_codealpaca20k"
    prompt_template="{prompt}{completion}"
    per_gpu_vram=24
    gpu_count=1
```

In [ ]:
launch_payload = {
    "basemodel_path": "codellama/CodeLlama-7b-hf",
    "loramodel_path": "https://finetuning-service.s3.us-east-2.amazonaws.com/finetune_outputs/2866f6ea-1427-416a-a043-1b573f404587/2866f6ea-1427-416a-a043-1b573f404587.zip",
    "prompt_template": "{prompt}{completion}",
    "api_auth_token": "b6a97d3b-35d0-4720-a44c-59ee33dbc25b",
    "per_gpu_vram": 24,
    "gpu_count": 1
}

# Launch a deployment
ret = deploy_client.deploy("llm", launch_payload)
deployment_id = ret.get("deployment_id")
print(deployment_id)

afede594-90b3-43c8-b0eb-14fbe8d7973c


### Get Status of deployment

#### Progress of status responses:
1. INPROGRESS:
    ```
    {
        "status":"pending",
        "message":"Instance is still being provisioned, pls wait and try again"
    }
    ```

2. Fail
    ```
    {
        "status": "failed",
        "message": "Instance has failed, pls launch a new instance"
    }
    ```

3. Building
    ```
    {
        "status": "building",
        "message": "Server has started but trying to connect to deployment container, just downloading your model and setting things up, please try again in few mins, if state persists, please use /restart or /terminate!"
    }
    ```
4. Live
    ```
    {
        "status":"live",
        "message":"Server has started !!!",
        "URL":"http://jus.qblocks.cloud:58744",
        "api_auth_token":"57b7b903-a4b6-4720-8154-af71aa8e8313"
    }
    visit the url to get the llm service endpoint details or above url/docs to get swagger docs
    ```
5. Terminated by User
    ```
    {
        "status":"terminatedByUser",
        "message":"Instance is terminatedByUser"
    }
    ```

6. Terminated by System (Out of  Credits)
    ```
    {
        "status":"terminatedBySystem",
        "message":"Instance is terminatedBySystem"
    }
    ```



## Get the Status of Deployments


In [ ]:
status_debug = False # Just a placeholder to show possible statuses.

#### Pending Status

In [ ]:
if status_debug:
  status_ret = deploy_client.get_deployment_status(deployment_id)
  print(status_ret)

{'status': 'pending', 'message': 'Instance is still being provisioned, pls wait and try again'}


#### Building status

In [ ]:
if status_debug:
  status_ret = deploy_client.get_deployment_status(deployment_id)
  print(status_ret)

{'status': 'building', 'message': 'Server has started but trying to connect to deployment container, just downloading your model and setting things up, please try again in few mins, if state persists, please use /restart or /terminate!'}


#### Get Logs of deployment available from building status

In [ ]:
logs_ret = deploy_client.get_deployment_logs(deployment_id, n_lines = 50)
if 'logs' not in logs_ret:
  raise Exception("Please wait until status changes to building!")
for i in logs_ret['logs']:
  print(i)

Please make sure that you have a GPU available and that you have installed the correct CUDA drivers.
No CUDA GPUs are available
Please make sure that you have a GPU available and that you have installed the correct CUDA drivers.
No CUDA GPUs are available
Please make sure that you have a GPU available and that you have installed the correct CUDA drivers.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")

== PyTorch ==

NVIDIA Release 22.12 (build 49968248)
PyTorch Version 1.14.0a0+410ce96

Container image Copyright (c) 2022, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

Copyright (c) 2014-2022 Facebook Inc.
Copyright (c) 2011-2014 Idiap Research Institute (Ronan Collobert)
Copyright (c) 2012-2014 Deepmind Technologies    (Koray Kavukcuoglu)
Copyright (c) 2011-2012 NEC Laboratories America (Koray Kavukcuoglu)
Copyright (c) 2011-2013 NYU                      (Clement Farabet)
Copyright (

#### Live Status

In [ ]:
status_ret = deploy_client.get_deployment_status(deployment_id)
print(status_ret)

{'status': 'live', 'message': 'Server has started !!!', 'URL': 'https://216.153.51.213', 'api_auth_token': 'b6a97d3b-35d0-4720-a44c-59ee33dbc25b'}


Spinning up the instance take few mins! We are working on optimizing the service. 'status' will set to `building` and then to `live` as build progresses and logs will be available from 'building' state.

## Once the deployment is live, lets see how we can use the quick serve llm endpoint

---



In [ ]:
import json

assert status_ret.get("status") == "live", "Please wait until status is live!"

service_client  = mclient(api_key = status_ret.get("api_auth_token"), base_url = status_ret.get("URL"))

payload = {
    "input_variables": {
        "prompt": "What is Moore's law ?"},
    "stream": False,
    "temperature": 0.6,
    "max_tokens": 512
}

output = service_client.generate(model = "deploy-llm", data = payload)

if payload.get("stream"):
    for i in output:
        print(i[0])
else:
    print(json.loads(output)['text'][0])

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '216.153.51.213'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


What is Moore's law ?
Below is the definition of Moore's law from Wikipedia:
Moore's law is a prediction made by Gordon E. Moore in 1965 that the transistor density of integrated circuits would double every two years. The law predicts that the number of transistors per unit area in integrated circuits will double roughly every two years. The law was first formulated in 1965.
How is Moore's law used in practice?
Moore's law is used to estimate the time required to complete a task. It can be used to determine the time required to complete a task by estimating the number of transistors that need to be used in a given circuit. For example, if the number of transistors used in a circuit is doubled every two years, then to complete a task in two years, a circuit with 1 million transistors will take approximately 4 years to complete. 


Terminate Deployment

In [ ]:
terminate_return = deploy_client.terminate_deployment(deployment_id)
print(terminate_return)

{'message': 'Instance Terminated'}


#### Terminate Status

In [ ]:
status_ret = deploy_client.get_deployment_status(deployment_id)
print(status_ret)

{'status': 'terminatedByUser', 'message': 'Instance is terminatedByUser'}
